# Setting up

try installing in terminal

In [1]:
# !conda install -c conda-forge pyomo
# !conda install -c conda-forge ipopt glpk

Assume we have two banks

- A: 0.03 for (0, 10000) -> 0.015 for (10000,50000) -> 0.05 for (50000, inf)
- B: 0.005 for (0, 100000) -> 0.05 for (100000, 500000) -> 0.005 for (500000, inf)

# Idea

For example A: 0.03 for (0, 10000) -> 0.015 for (10000,50000) -> 0.05 for (50000, inf)
For Bank A, formulate the amount of asset by $(\delta_{11},\delta_{12},\delta_{13})$ as follows:
$$
f_A(\delta_{11},\delta_{12},\delta_{13}) = 0.03\delta_{11} + 0.015\delta_{12} + 0.005\delta_{13}
$$
subject to
$$
10000 w_{11} \leq \delta_{11} \leq 10000\\
40000 w_{12} \leq \delta_{12} \leq 40000w_{11}\\
0 \leq \delta_{13} \leq Xw_{12}
$$
where $X$ is principal, and $w_{11}, w_{12} \in \{0,1\}$

The objective function is maximizing $f_A + f_B$

In [232]:
import pyomo.environ as pyo
import re

def getWeightName(inputDict):
    nameVars = []
    for bankName in inputDict:
        for i in range(len(inputDict[bankName][1])):
            nameVars.append(bankName+str(i+1))
    return nameVars

def getDeltaName(inputDict):
    nameVars = []
    for bankName in inputDict:
        for i in range(len(inputDict[bankName][1])+1):
            nameVars.append(bankName+str(i+1))
    return nameVars

def getDeltaLast(inputDict,principle):
    nameVars = []
    for bankName in inputDict:
        nameVars.append((bankName+str(len(inputDict[bankName][1])+1),principle))
    return nameVars

def getBankConstraint(inputDict):
    nameVars = []
    bounds   = []
    for bankName in inputDict:
        for i in range(len(inputDict[bankName][1])):
            nameVars.append(bankName+str(i+1))
        bounds += [x[0]-x[1] for x in zip(inputDict[bankName][1],[0] + inputDict[bankName][1][:-1])]
    return list(zip(nameVars, bounds))

def rule_lower(m, c, bound):
    return m.w[c]*bound <= m.delta[c]

def rule_upper(m, c, bound):
    if int(c[-1])-1 == 0:
        return m.delta[c] <= bound
    else:
        return m.delta[c] <= bound*m.w[c[:-1] + str(int(c[-1])-1)]
    
def rule_upper_last(m, c, principle):
    if int(c[-1])-1 == 0:
        return m.delta[c] <= principle
    else:
        return m.delta[c] <= principle*m.w[c[:-1] + str(int(c[-1])-1)]

def getInterestRate(inputDict):
    rate = []
    for bankName in inputDict:
        rate += inputDict[bankName][0]
    return rate

def printReport(m):
    print(f"Maximum interest = {pyo.value(m.interest)}")
    print(f"Average interest rate = {pyo.value(m.interest)/principle *100:.2f}%")
    print(':==============================:')
    asset_value = {}
    interest = {}
    for rate, bankName in zip(getInterestRate(inputDict),getDeltaName(inputDict)):
        key = re.match(r'(.+)(\d+)',bankName).group(1)
        asset_value[key] = asset_value.get(key,0) + pyo.value(m.delta[bankName])
        interest[key] = interest.get(key,0) + pyo.value(rate*m.delta[bankName])
    # print(asset_value)
    # print(interest)
    print(f'{"":10s} {"amount":^12s} {"interest":^12s}')
    for key in asset_value:
        print(f'{key:>10s}:{asset_value[key]:^12.0f} {interest[key]:^12.0f}')
    print(':==============================:')

given input: dictionary in the forms of

`{
    bankName: ([rate1, rate2, ..., rate(k+1)],[bound1, bound2, ..., bound(k)])
}`

In [ ]:
inputDict = {
    'LH':([0.0025, 0.0175,0.0555,0.015,0.0025,0],[100000, 900000, 1000000, 3000000, 100000000]),
    'KPP':([0.02,0.04,0.02,0.0155,0.005],[5000,10000,50000,1500000]),
    'Dime':([0.03,0.015,0.005],[10000,1000000]),
    'CIMBChill': ([0.005,0.018,0.0288,0.002],[10000,50000,100000]),
    'Kept':([0.0175],[]),
    'TTB':([0.022,0.016,0.012],[100000,1000000]),
    'Alpha':([0.02,0.007],[500000]),
    'CIMBSpeed':([0.008,0.0188],[100000]),
}

In [233]:
def optimize(principle,inputDict,verbose=False):
    m = pyo.ConcreteModel()

    m.w     = pyo.Var(getWeightName(inputDict),
                    initialize=0,within=pyo.Binary)
    m.delta = pyo.Var(getDeltaName(inputDict),
                    bounds=(0,principle))
    m.lower  = pyo.Constraint(getBankConstraint(inputDict),rule = rule_lower)
    m.upper  = pyo.Constraint(getBankConstraint(inputDict),rule = rule_upper)
    m.last   = pyo.Constraint(getDeltaLast(inputDict,principle),rule = rule_upper_last)
    m.principle   = pyo.Constraint(expr = sum(m.delta[x] for x in getDeltaName(inputDict)) == principle)
    m.interest = pyo.Objective(expr=sum(x[0]*m.delta[x[1]] for x in zip(getInterestRate(inputDict),getDeltaName(inputDict))),sense = pyo.maximize)
    pyo.SolverFactory("glpk").solve(m)
    
    if verbose:
        printReport(m)
        
    return m

m = optimize(100000,inputDict,True)

Maximum interest = 2360.0
Average interest rate = 0.05%
:==============================:
              amount      interest  
        LH:     0            0      
       KPP:   10000         300     
      Dime:   10000         300     
 CIMBChill:     0            0      
      Kept:     0            0      
       TTB:   80000         1760    
     Alpha:     0            0      
 CIMBSpeed:     0            0      
:==============================:
